# Signal Displays Verification Test

## Table of Contents\n
#### [0.1 - Global Configuration](#02---setup-test-equipment)
#### [0.2 - Setup Test Equipment](#02---setup-test-equipment)
#### [1 -> Test Steps](#1---test-steps)
This notebook provides for indepenent control of the test equipment during the rf signal displays verification testchain and channelisation test so that test equipment setup may be changed while the telescope is scanning.  Test steps are traceable to the jama test but only test steps that modify test equipment input parameters are included.

## 0.1 Global Configuration

### 0.1.1 Import dependencies

In [ ]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from datetime import datetime
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 0.1.2 Setup Global Variables and Configuration

In [ ]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

### 0.1.3 Test Connections to Namespaces

In [ ]:
test_equipment.smoke_test()

## 1. Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 0.2.1 Configure Test Equipment State 

In [ ]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 0.2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 0.2.3 Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

### 0.2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 0.2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

**Step 3:**

Use the standard noise inputs to the Digitisers of the SUT providing wideband noise at nominal level of -130 dBm/Hz, band-limited to give a recognisable shape to the results.  

Expected Result:  
*Sky Simulator dashboard reports correct values set.*

In [ ]:
# test equipment setup values - band 1
bottom_of_band = 350.0e6
top_of_band = 1050.0e6
sky_sim_loss = 35.0
frequency_to_set = bottom_of_band  #bottom of band
spfrx_power_level = -90.0   #to provide -90dbm at spfrx input
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator
siggen_power_level = spfrx_power_level + sky_sim_loss

# print current
timestr = datetime.now().strftime('%F %T.%f')[:-3]
print ("Current Date and Time: ", timestr)
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", siggen_power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(1)
timestr = datetime.now().strftime('%F %T.%f')[:-3]
print ("Current Date and Time: ", timestr)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print("Updated spfrx input power level:                       ", (signal_generator.power_dbm - sky_sim_loss))
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == siggen_power_level, print(
    f"Power level required is {siggen_power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

In [ ]:
# print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency:                     {signal_generator.frequency}")
print(f"Current signal generator power level:                   {signal_generator.power_dbm}")
print("Current spfrx input power level:                       ", (signal_generator.power_dbm - sky_sim_loss))
print(f"Current Sky Simulator Correlated Noise Source:          {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source:        {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band:                             {SSC.Band}")
print(f"Current Programmable attenuator current attenuation :   {prog_atten.channel_1}")


**Step 5:**

Sweep a CW tone over the frequency band at a level of -90 dBm (TBC). 

Expected Result:  
*Signal Generator displays CW tone sweeping over frequency band / view this on the Sky Sim Dashboard.*

In [ ]:
frequency_to_set = bottom_of_band 
time.sleep(0.1)
while frequency_to_set < top_of_band:
    frequency_to_set += 10.0e6
    signal_generator.write_attribute("frequency", frequency_to_set)
    time.sleep(2)
    assert signal_generator.frequency == frequency_to_set,print(
            f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
        )
    if frequency_to_set % 50e6 == 0:
        timestr = time.strftime("%Y %m %d-%H:%M:%S")
        print(timestr,f"Updated signal generator frequency : {signal_generator.frequency}")

Test code - control of siggen with scpi commands

In [19]:
from ska_mid_jupyter_notebooks.scripts import sig_gen_sweep

ModuleNotFoundError: No module named 'sg_smb100a_generate_frequency_sweep_2'

In [9]:
import telnetlib
smb100a = telnetlib.Telnet("10.165.3.1",5025)


In [10]:
output = smb100a.read_all()

KeyboardInterrupt: 